In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install bitsandbytes accelerate
!pip install gigachat
!pip install gigachain
!pip install langchain
!pip install langchain_gigachat
!pip install aiohttp

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import json
import csv
import sys
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

In [ ]:
# Порог для первичной фильтрации по косинусному score
MIN_SCORE = 0.5
auth = "NGE2NDUzZTMtYTMyZC00NjVkLTk0NTgtNzRjMWRhN2I1Y2RlOmMwNWE2MGM3LTBmMzgtNDY5MC1hZGFlLTliYjU2YjFiY2Y2YQ=="

llm = GigaChat(credentials=auth,
                model='GigaChat',
                verify_ssl_certs=False
                )

# Пусть вход и выход передаются как аргументы или заданы по умолчанию
INPUT_JSON  = "results_20250115_20250130_20250429_150253.json"
OUTPUT_JSON = "results_classified.json"
OUTPUT_CSV  = "results_classified.csv"

# 2) Читаем входной файл
with open(INPUT_JSON, encoding="utf-8") as f:
    articles = json.load(f)

output = []
for art in articles:
    score = art.get("score", 0.0)
    if score < MIN_SCORE:
        art["is_fraud"] = False
        art["type"] = "точно не мошенничество"
        art["comment"] = "низкий score"
        output.append(art)
        continue

    prompt = f"""
        Статья (similarity score={score:.3f}):
        Заголовок: {art['title']}

        {art['body']}

        Инструкция: Определи, относится ли эта статья к банковскому мошенничеству.
        Если да — укажи одну из тем:
          'телефонное мошенничество', 'кредитная афера',
          'инвестиционная пирамида', 'онлайн-мошенничество',
          'корпоративное мошенничество' и т.д.
        Если нет — ответь 'не мошенничество'.
        Выведи ТОЛЬКО ОДИН ВАЛИДНЫЙ JSON-объект БЕЗ ДОПОЛНИТЕЛЬНОГО ТЕКСТА:
        {{"is_fraud": true, "type": "...", "comment": "..."}}
        или
        {{"is_fraud": false, "type": "не мошенничество", "comment": "..."}}
        
        Пример правильного ответа:
        {{"is_fraud": true, "type": "онлайн-мошенничество", "comment": "В данной статье описываются действия мошенников, связанные с нелегальным получением денег через Интернет.т"}}
        """

    try:
        # Используем GigaChat вместо pipeline
        response_msg = llm([HumanMessage(content=prompt)])
        # В зависимости от версии LangChain может вернуться список или единичное сообщение
        resp = response_msg.content if hasattr(response_msg, 'content') else response_msg[0].content

        # Обработка и извлечение JSON из текста ответа
        json_str = resp.split("```")[1] if "```" in resp else resp
        json_str = json_str.replace("\\_", "_")

        start = json_str.find('{')
        end = json_str.rfind('}') + 1
        json_part = json_str[start:end].strip()

        # Приведём к валидному JSON
        json_part = (
            json_part
            .replace('«', '"').replace('»', '"')
            .replace('\n', ' ')
            .replace("'", '"')
            .replace('True', 'true').replace('False', 'false')
            .replace('|', '')
            .replace('is_fraud:', '"is_fraud":')
            .replace('type:', '"type":')
            .replace('comment:', '"comment":')
        )
        if '}{' in json_part:
            json_part = json_part.split('}{')[0] + '}'
        if json_part.count('{') > json_part.count('}'):
            json_part += '}'

        import json5
        info = json5.loads(json_part)

        for k in ('is_fraud','type','comment'):
            if k not in info:
                raise ValueError(f"Missing key {k}")

        info['is_fraud'] = bool(info['is_fraud'])
        info['type']     = str(info['type']).strip()[:50]
        info['comment']  = str(info['comment']).strip()

    except Exception as e:
        print(f"\nError: {e}")
        print("Raw response:", resp)
        info = {
            "is_fraud": False,
            "type":     "ошибка обработки",
            "comment":  f"Ошибка: {str(e)[:100]}"
        }

    art.update(info)
    output.append(art)

# 3) Сохраняем расширенный JSON
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

# 4) Сохраняем CSV
fieldnames = list(output[0].keys()) if output else []
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output)

print(f"Done: {OUTPUT_JSON}, {OUTPUT_CSV}")